In [16]:
import pandas as pd

def serve_data(ret=False, serve_local=False):
    global MASTER_ALL
    global MASTER_PID
    global DATE_MAPPER
    global KEY_VALUE

    if serve_local:
        MASTER_ALL = pd.read_pickle('Data/MASTER_ALL.pkl', compression='gzip')
        MASTER_PID = pd.read_pickle('Data/MASTER_PID.pkl', compression='gzip')

    else:
        print('Grabbing from S3')
        MASTER_ALL = pd.read_pickle(
            'https://jordansdatabucket.s3-us-west-2.amazonaws.com/covid19data/MASTER_ALL_NEW.pkl', compression='gzip')
        MASTER_PID = pd.read_pickle(
            'https://jordansdatabucket.s3-us-west-2.amazonaws.com/covid19data/MASTER_PID_NEW.pkl', compression='gzip')
        DATE_MAPPER = pd.DataFrame(MASTER_ALL['Date'].unique())
        KEY_VALUE = dict(zip(list(MASTER_PID.index), list(
            MASTER_PID['Text_Confirmed'].str.split('<br>').str.get(0))))#.str.replace('US', 'United States'))))
        KEY_VALUE = pd.DataFrame(list(KEY_VALUE.values()), index=KEY_VALUE.keys(), columns=['name'])

    if ret:
        return MASTER_ALL, MASTER_PID, DATE_MAPPER, KEY_VALUE
    else:
        return
serve_data()

Grabbing from S3


In [36]:
import plotly
mapbox_style = 'mapbox://styles/jwillis0720/ck89nznm609pg1ipadkyrelvb'
mapbox_access_token = open('./.mapbox_token').readlines()[0]
colors = plotly.colors.qualitative.Dark24
# remove black elment
colors.remove(colors[5])
# lets repeat it enough time sjust incase
colors = colors * 10

def get_closest_inerval(n, s='max'):
    import math
    d = math.floor(math.log(abs(n)) / math.log(10))
    scale = 10**d
    if s == 'max':
        return math.ceil(n/scale) * scale
    return math.floor(n/scale) * scale

def plot_map(dataframe, metrics, zoom, center):
    data_traces = []
    if not metrics or dataframe.empty:
        data_traces.append(go.Scattermapbox(
            lon=[],
            lat=[]
        ))
    if 'confirmed' in metrics and not dataframe.empty:
        # First Do Confirmed
        gb_confirmed = dataframe.groupby('CSize')

        # Do this so we can sort them if need be
        for group in gb_confirmed:
            plotting_df = group[1]
            sizes = plotting_df['confirmed']
            # This should be so, but okay
            sizes[sizes < 1] = 1
            name = "{}-{}".format(get_closest_inerval(int(plotting_df['confirmed'].min()), 'min'),
                                  get_closest_inerval(int(plotting_df['confirmed'].max()), 'max'))
            data = go.Scattermapbox(
                lon=plotting_df['lon'],
                lat=plotting_df['lat'],
                customdata=plotting_df['PID'],
                text=plotting_df['Text_Confirmed'],
                hoverinfo='text',
                name=name,
                mode='markers',
                marker=dict(
                    opacity=0.85,
                    sizemin=3,
                    size=plotting_df['CSize'],
                    color=plotting_df['CColor'],
                    sizemode='area'
                )
            )
            data_traces.append(data)
    if 'deaths' in metrics and not dataframe.empty:
        # Second Do Confirmed
        dataframe = dataframe[dataframe['deaths'] > 1]
        gb_deaths = dataframe.groupby('DSize')

        # Do this so we can sort them if need be
        for group in gb_deaths:
            plotting_df = group[1]
            # This should be so, but okay
            name = "{}-{}".format(get_closest_inerval(int(plotting_df['deaths'].min()), 'min'),
                                  get_closest_inerval(int(plotting_df['deaths'].max()), 'max'))
            data = go.Scattermapbox(
                lon=plotting_df['lon'],
                lat=plotting_df['lat'],
                customdata=plotting_df['PID'],
                text=plotting_df['Text_Deaths'],
                hoverinfo='text',
                name=name,
                mode='markers',
                marker=dict(
                    opacity=0.85,
                    sizemin=3,
                    size=plotting_df['DSize'],
                    color=plotting_df['DColor'],
                    sizemode='area'
                )
            )
            data_traces.append(data)
    layout = dict(
        autosize=True,
        showlegend=True,
        mapbox=dict(
            accesstoken=mapbox_access_token,
            style=mapbox_style,
            zoom=zoom,
            center=center
        ),
        hovermode="closest",
        margin=dict(r=0, l=0, t=0, b=0),
        dragmode="pan",
        legend=dict(
            x=0.00,
            y=0.00,
            orientation='v',
            traceorder="normal",
            font=dict(
                family="Montserrat",
                color="white",
                size=14
            ),
            bgcolor='rgba(0,0,0,0.6)'
        ),
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)'
    )
    return {'data': data_traces, 'layout': layout}

sub_df = MASTER_ALL.set_index('Date').loc[MASTER_ALL.set_index('Date').index[-2]]
go.Figure(plot_map(sub_df,['confirmed','deaths'],0.5, dict(lat=19.75, lon=-34.2))).show()

/Users/jordanwillis/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/jordanwillis/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:8765: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,country,state,county,lat,lon,pop,confirmed,deaths,country_code,granularity,...,confirmed_lower,deaths_upper,deaths_lower,Text_Confirmed,Text_Deaths,PID,CSize,DSize,CColor,DColor
Date,,,,,,,,,,,,,,,,,,,,,
2020-01-22,Afghanistan,,,33.0,65.0,2.840000e+07,0.0,0.0,AF,country,...,0.0,0.0,0.0,Afghanistan<br> Total Cases: 0,Afghanistan<br> Total Deaths: 0,0,0.000000,0.000000,white,white
2020-01-22,Albania,,,41.1533,20.1683,3.639453e+06,0.0,0.0,AL,country,...,0.0,0.0,0.0,Albania<br> Total Cases: 0,Albania<br> Total Deaths: 0,1,0.000000,0.000000,white,white
2020-01-22,Algeria,,,28.0339,1.6596,3.417819e+07,0.0,0.0,DZ,country,...,0.0,0.0,0.0,Algeria<br> Total Cases: 0,Algeria<br> Total Deaths: 0,2,0.000000,0.000000,white,white
2020-01-22,Andorra,,,42.5063,1.5218,8.388800e+04,0.0,0.0,AD,country,...,0.0,0.0,0.0,Andorra<br> Total Cases: 0,Andorra<br> Total Deaths: 0,3,0.000000,0.000000,white,white
2020-01-22,Angola,,,-11.2027,17.8739,1.279929e+07,0.0,0.0,AO,country,...,0.0,0.0,0.0,Angola<br> Total Cases: 0,Angola<br> Total Deaths: 0,4,0.000000,0.000000,white,white
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-01-22,Western Sahara,,,24.2155,-12.8858,5.000000e+05,0.0,0.0,EH,country,...,0.0,0.0,0.0,Western Sahara<br> Total Cases: 0,Western Sahara<br> Total Deaths: 0,2974,0.000000,0.000000,white,white
2020-01-22,Yemen,,,15.552726999999999,48.516388,2.382278e+07,0.0,0.0,YE,country,...,0.0,0.0,0.0,Yemen<br> Total Cases: 0,Yemen<br> Total Deaths: 0,2975,0.000000,0.000000,white,white
2020-01-22,Zambia,,,-15.4167,28.2833,1.186274e+07,0.0,0.0,ZM,country,...,0.0,0.0,0.0,Zambia<br> Total Cases: 0,Zambia<br> Total Deaths: 0,2976,0.000000,0.000000,white,white
